In [3]:
from haystack import Document, component
from haystack.components.builders import PromptBuilder
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.generators.openai_utils import _convert_message_to_openai_format
from haystack.dataclasses import ChatMessage, StreamingChunk
from haystack.document_stores.in_memory import InMemoryDocumentStore
# from haystack_integrations.components.embedders.cohere import CohereDocumentEmbedder, CohereTextEmbedder
from haystack.components.embedders import OpenAITextEmbedder, OpenAIDocumentEmbedder
from haystack_integrations.document_stores.chroma import ChromaDocumentStore

from datasets import load_dataset
from openai import OpenAI, Stream
from openai.types.chat import ChatCompletion, ChatCompletionChunk
from typing import List, Any, Dict, Optional, Callable, Union
from pydantic import BaseModel



In [ ]:


document_store = ChromaDocumentStore(persist_path="got_chroma_db")

dataset = load_dataset("Tuana/game-of-thrones", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]

doc_embedder = OpenAIDocumentEmbedder(model="text-embedding-3-small")
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])

In [6]:
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever

# Initialize components
# retriever = InMemoryEmbeddingRetriever(document_store=document_store, top_k=3)  # Limit to top 3 results
document_store = ChromaDocumentStore(persist_path="got_chroma_db")
retriever = ChromaEmbeddingRetriever(document_store=document_store, top_k=3)
text_embedder = OpenAITextEmbedder(model="text-embedding-3-small")
generator = OpenAIGenerator()

# Build prompt template
template = """Answer the question based on the provided context and chat history. If you cannot find the answer in the context, say so.

Previous conversation:
{% for message in chat_history %}
{% if message.role == 'user' %}User: {{ message.content }}{% endif %}
{% if message.role == 'assistant' %}Assistant: {{ message.content }}{% endif %}
{% endfor %}

Context:
{% for doc in documents %}
{{ doc.content }}
{% endfor %}

Question: {{question}}

Answer:"""

prompt_builder = PromptBuilder(template=template)

class ChatSession:
    def __init__(self):
        self.chat_history = []
        self.context_history = []
        
    def chat(self, question: str):
        # Embed question and retrieve relevant docs
        embedded_query = text_embedder.run(question)
        retrieved_docs = retriever.run(embedded_query["embedding"])
        
        # Store context
        self.context_history.extend(retrieved_docs["documents"])
        
        # Build prompt with retrieved context and chat history
        prompt_params = {
            # "documents": retrieved_docs["documents"], 
            "documents": self.context_history,
            "question": question,
            "chat_history": self.chat_history
        }
        prompt = prompt_builder.run(**prompt_params)
        
        # Generate answer
        response = generator.run(prompt["prompt"])
        answer = response["replies"][0]
        
        # Update chat history
        self.chat_history.append(ChatMessage.from_user(question))
        self.chat_history.append(ChatMessage.from_assistant(answer))
        
        return f"{answer}\n\nSources: {', '.join(list(set([i.meta["name"] for i in session.context_history])))}" #output the document names


In [ ]:

session = ChatSession()

while True:
    message = input("Ask a question about Game of Thrones (type 'exit' to quit): ")
    if message.lower() == 'exit':
        print("Exiting the chat session.")
        break
    response = session.chat(message)
    print(f"Assistant: {response}\n")


In [58]:
# Gradio chat interface
import gradio as gr

session = ChatSession()

def chat_response(message, history):
    return session.chat(message)

demo = gr.ChatInterface(
    fn=chat_response,
    title="Game of Thrones Chat", 
    description="Ask questions about Game of Thrones",
)

demo.launch(show_api=False, share=False)



['30_List_of_A_Song_of_Ice_and_Fire_characters.txt',
 '43_Arya_Stark.txt',
 '371_Cersei_Lannister.txt']

In [ ]:
write a 3 paragraph summary of aria starks personality and drivers

how has her sister influenced her?

who influenced her more (both positive and negative) sansa or the hound?

a choice between the two